In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
plt.rc('text', usetex=True)
plt.rc('font', family='Serif')

import matplotlib as mpl 
mpl.rcParams['figure.figsize'] = [10, 7]
mpl.rcParams['font.size'] = 27

mpl.rcParams['savefig.dpi'] = 150 
mpl.rcParams['xtick.minor.visible'] = True
mpl.rcParams['ytick.minor.visible'] = True
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'

mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True

mpl.rcParams['xtick.major.size'] = 10 
mpl.rcParams['xtick.minor.size'] = 5 

mpl.rcParams['ytick.major.size'] = 10 
mpl.rcParams['ytick.minor.size'] = 5 

mpl.rcParams['xtick.labelsize'] = 23
mpl.rcParams['ytick.labelsize'] = 23

In [2]:
M55_path = 'C:\\Users\\marti\\Desktop\\school work\\Dartmouth\\GC_ages\\M55'

In [15]:
AS_center = [3005.49976,2997.75391]
OBS_center = [2995.02393,3020.84351]
V_diff=30.98
I_diff = V_diff-0.737

In [4]:
M55_fitstars = pd.read_csv("{}\\M55_fitstars.dat".format(M55_path),sep=',')
M55_fitstars

,v,v_err,i,i_err,vi,vi_err,x,y
0,18.630,0.0038,18.052,0.0041,0.578,0.0056,2278.684,982.716
1,18.809,0.0041,18.222,0.0044,0.587,0.0061,2451.712,968.601
2,18.635,0.0038,18.062,0.0041,0.573,0.0056,2754.560,883.018
3,18.944,0.0044,18.355,0.0047,0.589,0.0064,2765.354,985.549
4,19.221,0.0120,18.585,0.0010,0.636,0.0120,2913.994,974.980
...,...,...,...,...,...,...,...,...
6125,18.580,0.0035,18.006,0.0038,0.574,0.0052,3475.666,5046.050
6126,19.111,0.0045,18.497,0.0048,0.614,0.0066,3541.681,5008.010
6127,18.051,0.0028,17.503,0.0030,0.548,0.0041,3652.334,5007.633
6128,18.341,0.0032,17.788,0.0034,0.553,0.0047,3663.224,5005.632


In [31]:
def find_fit_val(arr,x):
    for idx in range(len(arr)):
        if x < arr[idx]:
            return arr[idx]
    return arr[-1]

In [32]:
df_V_bound = pd.read_csv("{}\\inputfiles\\VerrBoundary.dat".format(M55_path),sep='\s+',skiprows=3, names=['Rad', 'Mag', 'Nstar', 'Completness'])
Rad_bounds = np.sort(np.unique(df_V_bound['Rad'].values))
V_mag_bounds = np.sort(np.unique(df_V_bound['Mag'].values))
rad = np.sqrt((M55_fitstars['x'].values - OBS_center[0])**2 + (M55_fitstars['y'].values - OBS_center[1])**2)

In [29]:
V_bin = []
for i in range(len(M55_fitstars)):
    min_V=find_fit_val(V_mag_bounds,M55_fitstars['v'].values[i] - V_diff)
    min_r = find_fit_val(Rad_bounds,rad[i])
    for j in range(len(df_V_bound)):
        if (np.abs(df_V_bound['Rad'].values[j] - min_r) < 0.01) & (np.abs(df_V_bound['Mag'].values[j] - min_V) < 0.01):
            V_bin.append(j+1)
            break

In [33]:
df_I_bound = pd.read_csv("{}\\inputfiles\\IerrBoundary.dat".format(M55_path),sep='\s+',skiprows=3, names=['Rad', 'Mag', 'Nstar', 'Completness'])
Rad_bounds = np.sort(np.unique(df_I_bound['Rad'].values))
I_mag_bounds = np.sort(np.unique(df_I_bound['Mag'].values))
rad = np.sqrt((M55_fitstars['x'].values - OBS_center[0])**2 + (M55_fitstars['y'].values - OBS_center[1])**2)

In [34]:
I_bin = []
for i in range(len(M55_fitstars)):
    min_I = find_fit_val(I_mag_bounds,M55_fitstars['i'].values[i] - I_diff)
    min_r = find_fit_val(Rad_bounds,rad[i])
    for j in range(len(df_I_bound)):
        if (np.abs(df_I_bound['Rad'].values[j] - min_r) < 0.01) & (np.abs(df_I_bound['Mag'].values[j] - min_I) < 0.01):
            I_bin.append(j+1)
            break

In [38]:
M55_fitstars['Vbin'] = V_bin
M55_fitstars['Ibin'] = I_bin

In [39]:
M55_fitstars

,v,v_err,i,i_err,vi,vi_err,x,y,Vbin,Ibin
0,18.630,0.0038,18.052,0.0041,0.578,0.0056,2278.684,982.716,70,70
1,18.809,0.0041,18.222,0.0044,0.587,0.0061,2451.712,968.601,80,80
2,18.635,0.0038,18.062,0.0041,0.573,0.0056,2754.560,883.018,70,70
3,18.944,0.0044,18.355,0.0047,0.589,0.0064,2765.354,985.549,79,79
4,19.221,0.0120,18.585,0.0010,0.636,0.0120,2913.994,974.980,79,79
...,...,...,...,...,...,...,...,...,...,...
6125,18.580,0.0035,18.006,0.0038,0.574,0.0052,3475.666,5046.050,70,70
6126,19.111,0.0045,18.497,0.0048,0.614,0.0066,3541.681,5008.010,80,80
6127,18.051,0.0028,17.503,0.0030,0.548,0.0041,3652.334,5007.633,60,60
6128,18.341,0.0032,17.788,0.0034,0.553,0.0047,3663.224,5005.632,70,70


In [40]:
M55_fitstars.to_csv("{}\\M55_fitstars_with_bins.dat".format(M55_path))